In [44]:
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [56]:
df = pd.read_csv('geo_data_0.csv')
df1 = pd.read_csv('geo_data_1.csv')
df2 = pd.read_csv('geo_data_2.csv')

In [58]:
df.sample(20)
df[df['id'].duplicated()]
df[df['id'] == 'HZww2']


,id,f0,f1,f2,product
931,HZww2,0.755284,0.368511,1.863211,30.681774
7530,HZww2,1.061194,-0.373969,10.430210,158.828695


In [59]:
df1.sample(20)
df1[df1['id'].duplicated()]
df1[df1['id'] == 'LHZR0']

,id,f0,f1,f2,product
1305,LHZR0,11.170835,-1.945066,3.002872,80.859783
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625


In [60]:
df2.sample(20)
df2[df2['id'].duplicated()]
df2[df2['id'] == 'xCHr8']

,id,f0,f1,f2,product
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641


Se han examinado los 3 archivos csv y al parecer no hay ningun problema con ninguno, y los ids duplicados que hay es por la diferencia de region. Por lo tanto parecen estar listos para lo siguiente.

In [48]:
features = df[['f0', 'f1', 'f2']]
target = df['product']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

model =LinearRegression() # inicializa el constructor de modelos
model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
predictions_valid = model.predict(features_valid) # obtén las predicciones del modelo en el conjunto de validación

result =mean_squared_error(target_valid, predictions_valid)**0.5 # calcula la RECM en el conjunto de validación
print("RECM del modelo de regresión lineal en el conjunto de validación:", result)
mean_prediction = predictions_valid.mean()
print("Volumen medio de las reservas: ", mean_prediction)

RECM del modelo de regresión lineal en el conjunto de validación: 37.5794217150813
Volumen medio de las reservas:  92.59256778438035


En lo anterior dividi los datos para el conjunto de entrenamiento y el de validacion, y tengo las respuestas validas en target_valid y las predicciones en predictions_valid, tambien se calculo el rmse y el volumen medio de las reservas.

El analisis de esta parte es el siguiente:

RECM (Raíz Cuadrada del Error Cuadrático Medio):
RECM: 37.58
La RECM de aproximadamente 37.58 indica el error promedio entre las reservas reales y las predichas por el modelo.
A menor valor de RECM, mejor es el rendimiento del modelo en cuanto a precisión de las predicciones.
Volumen Medio de las Reservas Predicho:
Volumen medio: 92.59 
El volumen medio de reservas predicho es de aproximadamente 92.59. Este es el promedio esperado basado en las características (f0, f1, f2) utilizadas en el modelo.

Ahora procedere a hacer todo esto en funciones para usarlas con los 3 archivos. Tambien hacer la comparacion del volumen medio con el umbral que se menciona en el proyecto, al mismo tiempo estoy mostrando los 200 pozoz con valores mas altos por region y los muestro utilizando un bucle for.

In [76]:
# Función 1: Dividir en Conjuntos
def split_data(df, target_col):
    features = df[['f0', 'f1', 'f2']]
    target = df['product']
    return train_test_split(features, target, test_size=0.25, random_state=12345)

# Función 2: Entrenar Modelo
def train_model(features_train, target_train):
    model = LinearRegression()
    model.fit(features_train, target_train)
    return model

# Función 3: Evaluar Modelo
def evaluate_model(model, features_valid, target_valid):
    predictions = model.predict(features_valid)
    rmse = mean_squared_error(target_valid, predictions) ** 0.5
    mean_prediction = predictions.mean()
    return rmse, mean_prediction

# Función 4: Comparar volumen medio con el umbral
def compare_to_threshold(mean_prediction, threshold=111.1):
    if mean_prediction >= threshold:
        return f"El volumen medio de reservas ({mean_prediction}) es suficiente para evitar pérdidas."
    else:
        return f"El volumen medio de reservas ({mean_prediction}) es insuficiente para evitar pérdidas."

# Función 5: Seleccionar los mejores 200 pozos de todas las zonas
def select_top_pozos(df, top_n=200):
    df_sorted = df.sort_values(by='product', ascending=False)
    return df_sorted.head(top_n)

# Aplicar la función a cada región
top_pozos_region1 = select_top_pozos(df)
top_pozos_region2 = select_top_pozos(df1)
top_pozos_region3 = select_top_pozos(df2)

# Diccionario de regiones y sus datos
regiones = {
    'Región 1': top_pozos_region1,
    'Región 2': top_pozos_region2,
    'Región 3': top_pozos_region3
}

# Calcular el volumen total para cada región
total_volume_region1 = sum(top_pozos_region1['product'])
total_volume_region2 = sum(top_pozos_region2['product'])
total_volume_region3 = sum(top_pozos_region3['product'])

volumenes = {
    'total_volumen_region1': total_volume_region1,
    'total_volumen_region2': total_volume_region2,
    'total_volumen_region3': total_volume_region3
}

# Bucle para mostrar los resultados de regiones
for region, data in regiones.items():
    print(f"\n{region}:")
    print(data[['id', 'product']])

# Bucle para mostrar los resultados de los volúmenes
for volumen, data in volumenes.items():
    print(f"\n{volumen}: {data}")

# Procesamiento para la región 1
features_train, features_valid, target_train, target_valid = split_data(df, 'product')
model = train_model(features_train, target_train)
rmse, mean_prediction = evaluate_model(model, features_valid, target_valid)
print("Volumen medio de las reservas: ", mean_prediction)
print(f"RMSE for geo_data_1: {rmse}")
print(compare_to_threshold(mean_prediction))

# Procesamiento para la región 2
features_train_1, features_valid_1, target_train_1, target_valid_1 = split_data(df1, 'product')
model_1 = train_model(features_train_1, target_train_1)
rmse_1, mean_prediction_1 = evaluate_model(model_1, features_valid_1, target_valid_1)
print("Volumen medio de las reservas: ", mean_prediction_1)
print(f"RMSE for geo_data_1: {rmse_1}")
print(compare_to_threshold(mean_prediction_1))

# Procesamiento para la región 3
features_train_2, features_valid_2, target_train_2, target_valid_2 = split_data(df2, 'product')
model_2 = train_model(features_train_2, target_train_2)
rmse_2, mean_prediction_2 = evaluate_model(model_2, features_valid_2, target_valid_2)
print("Volumen medio de las reservas: ", mean_prediction_2)
print(f"RMSE for geo_data_1: {rmse_2}")
print(compare_to_threshold(mean_prediction_2))



Región 1:
          id     product
8826   rjMou  185.364347
99818  7cHIv  185.362690
94175  uCDzR  185.355615
1925   IfqrC  185.354980
45291  5FEPb  185.352015
...      ...         ...
34856  Xqm8i  184.366474
54135  oAHG7  184.362338
63598  CaN8V  184.360762
4941   orIzp  184.358756
79494  obTWo  184.356455

[200 rows x 2 columns]

Región 2:
          id     product
89365  kM3uQ  137.945408
7215   oU4Lm  137.945408
79168  tkegj  137.945408
79174  1ZlMX  137.945408
79187  XjkaY  137.945408
...      ...         ...
63977  vpuvk  137.945408
98993  MIHmR  137.945408
98995  zkon9  137.945408
98999  DxgRk  137.945408
99003  o6Xz2  137.945408

[200 rows x 2 columns]

Región 3:
          id     product
79705  UAhji  190.029838
93444  IB0JE  190.013589
35099  2HeCn  190.011722
21943  dldNH  190.010982
37870  CpuBi  190.010029
...      ...         ...
96482  hfbIz  189.038807
53814  6YAZG  189.038746
5138   RO6MB  189.037389
91855  y3w0m  189.024650
51559  inKEw  189.024253

[200 rows x 2 colu